# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint



# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citydata_df=pd.read_csv("../output_data/cities.csv")
citydata_df.dropna(inplace=True)
citydata_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rivers,50.0308,-100.2403,23.95,83,39,7.29,CA,1650905698
1,1,mar del plata,-38.0023,-57.5575,66.22,68,0,13.00,AR,1650905477
2,2,ushuaia,-54.8000,-68.3000,40.66,87,100,19.57,AR,1650905698
3,3,pringsewu,-5.3582,104.9744,74.46,97,75,3.02,ID,1650905699
4,4,port alfred,-33.5906,26.8910,60.19,81,50,4.00,ZA,1650905653


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#store latitude and longitude in locations
locations=citydata_df[["Lat","Lng"]]
#fill NaN values and  convert to float
humidity=citydata_df["Humidity"]
max_humi=max(humidity)
max_humi

100

In [4]:
#Plot Heatmap
gmaps.configure(api_key=g_key)
fig=gmaps.figure()

#create heat layer
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=max_humi,point_radius=1)
#Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame:
weather_df=citydata_df.loc[(citydata_df["Max Temp"]>70) & (citydata_df["Max Temp"]<80)& (10>citydata_df["Wind Speed"]) & (citydata_df["Cloudiness"]==0)]
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,guerrero negro,27.9769,-114.0611,74.84,33,0,4.70,MX,1650905736
132,132,vallenar,-28.5708,-70.7581,78.31,28,0,6.06,CL,1650905761
192,192,castro,-24.7911,-50.0119,77.04,49,0,3.71,BR,1650905790
196,196,umm lajj,25.0213,37.2685,78.73,48,0,6.22,SA,1650905792
254,254,saint-pierre,-21.3393,55.4781,74.88,75,0,8.05,RE,1650905825
291,291,tura,25.5198,90.2201,75.07,53,0,5.70,IN,1650905843
323,323,laguna,38.4210,-121.4238,70.09,48,0,4.61,US,1650905856
359,359,miguel auza,24.2933,-103.4503,75.20,26,0,7.87,MX,1650905873
494,494,safaga,26.7292,33.9365,79.03,48,0,3.80,EG,1650905936
536,536,san quintin,30.4833,-115.9500,74.12,26,0,5.10,MX,1650905956


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=weather_df.copy()
hotel_df.dropna(inplace=True)
hotel_df["Hotel Name"]=""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
73,73,guerrero negro,27.9769,-114.0611,74.84,33,0,4.70,MX,1650905736,
132,132,vallenar,-28.5708,-70.7581,78.31,28,0,6.06,CL,1650905761,
192,192,castro,-24.7911,-50.0119,77.04,49,0,3.71,BR,1650905790,
196,196,umm lajj,25.0213,37.2685,78.73,48,0,6.22,SA,1650905792,
254,254,saint-pierre,-21.3393,55.4781,74.88,75,0,8.05,RE,1650905825,
291,291,tura,25.5198,90.2201,75.07,53,0,5.70,IN,1650905843,
323,323,laguna,38.4210,-121.4238,70.09,48,0,4.61,US,1650905856,
359,359,miguel auza,24.2933,-103.4503,75.20,26,0,7.87,MX,1650905873,
494,494,safaga,26.7292,33.9365,79.03,48,0,3.80,EG,1650905936,
536,536,san quintin,30.4833,-115.9500,74.12,26,0,5.10,MX,1650905956,


In [7]:
params={"radius":5000,
        "type":"lodging",
       "keyword":"hotel",
       "key":g_key}
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url,params=params)
    response=response.json()
    print(f'Index:{index}, City Name:{hotel_df.loc[index,"City"]} ')
# pprint(response)
    try:
        hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
        print(f'The neareast hotel is {hotel_df.loc[index,"Hotel Name"]}')
    except:
        print("Missing field/result...skipping")


Index:73, City Name:guerrero negro 
The neareast hotel is The Halfway Inn
Index:132, City Name:vallenar 
The neareast hotel is Hotel Cecil
Index:192, City Name:castro 
The neareast hotel is Central Palace Hotel
Index:196, City Name:umm lajj 
The neareast hotel is Juman Hotel
Index:254, City Name:saint-pierre 
The neareast hotel is Hôtel Le Saint-Pierre
Index:291, City Name:tura 
The neareast hotel is Hotel Polo Orchid
Index:323, City Name:laguna 
The neareast hotel is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5
Index:359, City Name:miguel auza 
The neareast hotel is Hoteles Pueblo Inn
Index:494, City Name:safaga 
The neareast hotel is Nemo Dive Club & Hotel
Index:536, City Name:san quintin 
The neareast hotel is Old Mill Hotel


In [8]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
73,73,guerrero negro,27.9769,-114.0611,74.84,33,0,4.70,MX,1650905736,The Halfway Inn
132,132,vallenar,-28.5708,-70.7581,78.31,28,0,6.06,CL,1650905761,Hotel Cecil
192,192,castro,-24.7911,-50.0119,77.04,49,0,3.71,BR,1650905790,Central Palace Hotel
196,196,umm lajj,25.0213,37.2685,78.73,48,0,6.22,SA,1650905792,Juman Hotel
254,254,saint-pierre,-21.3393,55.4781,74.88,75,0,8.05,RE,1650905825,Hôtel Le Saint-Pierre
291,291,tura,25.5198,90.2201,75.07,53,0,5.70,IN,1650905843,Hotel Polo Orchid
323,323,laguna,38.4210,-121.4238,70.09,48,0,4.61,US,1650905856,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
359,359,miguel auza,24.2933,-103.4503,75.20,26,0,7.87,MX,1650905873,Hoteles Pueblo Inn
494,494,safaga,26.7292,33.9365,79.03,48,0,3.80,EG,1650905936,Nemo Dive Club & Hotel
536,536,san quintin,30.4833,-115.9500,74.12,26,0,5.10,MX,1650905956,Old Mill Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
figure_layout={
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)

In [11]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…